In [1]:
import numpy as np

## Read the dataset

### N = 50 / S = 15
1. Rate : 10 tx/sec 
 - Est system throughput  483.374196
 - Total transactions  28974
 - Peak throughput  496
 - Median round latency  0.014953374
 - Median all received latency  0.3381112
2. Rate: 100 tx/sec 
 - Est system throughput  727.14732392
 - Total transactions  43628
 - Peak throughput  786
 - Median round latency  0.20268750190
 - Median all received latency  0.4679472446
3. Rate: 1,000 tx/sec 
 - Est system throughput  781.9564927625192
 - Total transactions  46920
 - Peak throughput  812
 - Median round latency  0.2086963653564453
 - Median all received latency  0.4732102155685425 
4. 10,000 tx/sec
 - Est system throughput  768.1905648069853
 - Total transactions  46090
 - Peak throughput  837
 - Median round latency  0.20660734176635742
 - Median all received latency  0.4702136516571045
5. 100,000 tx/sec
 - Est system throughput  753.9773295613762
 - Total transactions  45238
 - Peak throughput  880
 - Median round latency  0.21443533897399902
 - Median all received latency  0.48249876499176025

### Resources update 

1. R = 10, N = 50

 - Est system throughput  807.2624085842427
 - Total transactions  48434
 - Peak throughput  895
 - Median round latency  0.20511221885681152
 - Median all received latency  0.47864699363708496

2. R = 15, N = 50

 - Est system throughput  859.4521186939215
 - Total transactions  51564
 - Peak throughput  900
 - Median round latency  0.19453024864196777
 - Median all received latency  0.4605334997177124
 
3. R = 20 

 - Est system throughput  857.5430546462355
 - Total transactions  51450
 - Peak throughput  898
 - Median round latency  0.17993402481079102
 - Median all received latency  0.4385809898376465

4. R = 30 

 - Est system throughput  911.1047533273896
 - Total transactions  54664
 - Peak throughput  958
 - Median round latency  0.15656065940856934
 - Median all received latency  0.40256381034851074
 
 rate 10,000
    Est system throughput  964.7011632353781
    Total transactions  57880
    Peak throughput  1017
    Median round latency  0.1573479175567627
    Median all received latency  0.4068477153778076

N = 50, S = 15, R = 30 
   - Est system throughput  1068.812031678148
   - Total transactions  64124
   - Peak throughput  1092
   - Median round latency  0.18102359771728516
   - Median all received latency  0.34412384033203125
    
### R = 30, S = 15, different N 

* N = 100
 - Est system throughput  912.2689528891686
 - Total transactions  54734
 - Peak throughput  1105
 - Median round latency  0.3490002155303955
 - Median all received latency  0.628920316696167
 
 Rate higher:
         Est system throughput  943.2793806520752
         - Total transactions  56594
        - Peak throughput  1140
        Median round latency  0.3628702163696289
        Median all received latency  0.6363089084625244
 
* N = 200

Est system throughput  973.7071922983016
Total transactions  58420
Peak throughput  1275
Median round latency  0.708975076675415
Median all received latency  1.0246912240982056

* N = 30 

Est system throughput  599.6550164093476
Total transactions  35970
Peak throughput  621
Median round latency  0.13649439811706543
Median all received latency  0.3757057189941406

* N = 30 more resoruces 

Est system throughput  608.6764387428414
Total transactions  36518
Peak throughput  620
Median round latency  0.1367044448852539
Median all received latency  0.37877726554870605

## Different graph structure

S = 15


S = 30 

Est system throughput  849.00242717158
Total transactions  50938
Peak throughput  918
Median round latency  0.35129213333129883
Median all received latency  0.662987232208252

S = 50 

Est system throughput  775.7466468245492
Total transactions  46544
Peak throughput  833
Median round latency  0.3449583053588867
Median all received latency  0.6616991758346558

S = 100

Est system throughput  686.8597121297582
Total transactions  41293
Peak throughput  737
Median round latency  0.40673553943634033
Median all received latency  0.8253422975540161



In [73]:
import ast
import csv

graph_path = "/home/bulat/project/DetectionExperiments/noodles/Datasets/leader_blocks_time_agg_f_5_n_100_r_40_asyc_2.csv"

peer_counts ={}
tx_seen= dict()
tx_stats = dict()
min_time = None
max_time = None
tx_map = dict()
# time,transaction,type,seq_num,seen_by

index = 0                        
with open(graph_path) as read_file:
    csv_reader = csv.reader(read_file)
    first = True
    for row in csv_reader:
        if first:
            first = False
        else:
            time = float(row[0])
            peer_id = int(row[-1])
            type_val = row[2]
            seq_num_tuple = ast.literal_eval(row[3])
            tx = ast.literal_eval(row[1])  
            if str(tx) not in tx_map:
                tx_map[str(tx)] = index
                index+=1

            tx_map_ind = tx_map[str(tx)]
            from_peer = int(tx['peer']) if 'peer' in tx else int(tx['from_peer'])
            to_peer = int(tx['to_peer']) if 'to_peer' in tx else None
            tx_id = str(tx_map_ind)+str(seq_num_tuple)

            if not min_time or time < min_time:
                min_time = time
            if not max_time or time > max_time:
                max_time = time 
            
            # Transaction seen by how many times
            if tx_id not in tx_seen:
                tx_seen[tx_id] = 1
                if tx_map_ind not in tx_stats:
                    tx_stats[tx_map_ind] = dict()
            else: 
                tx_seen[tx_id] += 1

            # Init peer info
            if peer_id not in peer_counts:
                peer_counts[peer_id] = {"from_count": 0, "to_count":0, "others":0}
            
            #print(from_peer, peer_id)
            #print(seq_num_tuple)
            #print(tx_map_ind)
            
            
            if from_peer == peer_id:
                # If this is a source transaction
                peer_counts[peer_id]["from_count"]+=1
                if int(seq_num_tuple[1]) == 0 and 'first_create' not in tx_stats[tx_map_ind]:
                    # This is creation block
                    tx_stats[tx_map_ind]['first_create'] = time
                elif int(seq_num_tuple[1]) != 0 and 'round_time' not in tx_stats[tx_map_ind]:
                    # The source peer sees the claim confirmation 
                    tx_stats[tx_map_ind]['round_time'] = time
            elif to_peer == peer_id:
                # Dest transaction 
                peer_counts[peer_id]["to_count"]+=1
                if int(seq_num_tuple[1]) == 0 and 'first_seen' not in tx_stats[tx_map_ind]:
                    # Spend first seen by the counterparty 
                    tx_stats[tx_map_ind]['first_seen'] = time
                elif int(seq_num_tuple[1]) != 0 and 'claim_time' not in tx_stats[tx_map_ind]:
                    # The transaction claimed 
                    tx_stats[tx_map_ind]['claim_time'] = time
            else:
                # Other transactions
                peer_counts[peer_id]["others"]+=1
                if 'last_time' not in tx_stats[tx_map_ind]:
                    tx_stats[tx_map_ind]['last_time'] = time
                elif tx_stats[tx_map_ind]['last_time'] < time:
                    tx_stats[tx_map_ind]['last_time'] = time

In [74]:
import math
import numpy as np

latency_round = []
latency_all = []
throughput = {l:0 for l in range(int(max_time)+1)}
errs = []

for t in tx_stats:
    if 'round_time' not in tx_stats[t]:
        pass
    else:
        val = tx_stats[t]
        if 'first_seen' not in val:
            errs.append(t)
            continue
        round_trip = abs(val['round_time'] - val['first_create'])
        latency_round.append(round_trip)
        throughput[math.floor(val['first_seen'])] += 1      
        if 'last_time' not in val:
            val['last_time'] = val['round_time']
        all_seen = abs(val['last_time'] - val['first_seen']) 
        latency_all.append(all_seen)

        if all_seen > 10:
            print(t, tx_stats[t])
            

In [75]:
# 100 nodes /  fanout 5 / R = 40 / async 2 send 
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  3891.981840471668
Total transactions  233502
Peak throughput  4288
Median round latency  0.24185621738433838
Median all received latency  0.5480101108551025
Errors  0


In [72]:
# 100 nodes /  fanout 5 / R = 40 / async send 
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  3943.5954278907275
Total transactions  236592
Peak throughput  4345
Median round latency  0.18439137935638428
Median all received latency  0.5066524744033813
Errors  0


In [68]:
# 100 nodes /  fanout 5 / R = 40
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  3911.856069858355
Total transactions  234674
Peak throughput  4491
Median round latency  0.23033738136291504
Median all received latency  0.31153154373168945
Errors  0


In [65]:
# 50 nodes /  fanout 5 / R = 50
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  3656.095480544004
Total transactions  219356
Peak throughput  3756
Median round latency  0.11726260185241699
Median all received latency  0.16791462898254395
Errors  0


In [62]:
# 50 nodes /  fanout 5 / R = 40
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  3535.7364646824453
Total transactions  212132
Peak throughput  3682
Median round latency  0.1298213005065918
Median all received latency  0.17923486232757568
Errors  0


In [59]:
# 50 nodes /  fanout 5 / R = 30
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  3496.3397413972193
Total transactions  209774
Peak throughput  3706
Median round latency  0.12245368957519531
Median all received latency  0.17805266380310059
Errors  0


In [56]:
# 50 nodes /  fanout 5 / R = 20
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  3443.0818678722458
Total transactions  206568
Peak throughput  3540
Median round latency  0.1265873908996582
Median all received latency  0.1883256435394287
Errors  0


In [53]:
# 50 nodes /  fanout 5 / R = 10
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  3023.5318637244477
Total transactions  181406
Peak throughput  3141
Median round latency  0.15140247344970703
Median all received latency  0.2160114049911499
Errors  0


In [50]:
# 300 nodes /  fanout 1
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  10626.802244326871
Total transactions  637300
Peak throughput  12027
Median round latency  0.08565771579742432
Median all received latency  0.06370317935943604
Errors  0


In [47]:
# 200 nodes /  fanout 1
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  10106.970190858412
Total transactions  598806
Peak throughput  11434
Median round latency  0.060135722160339355
Median all received latency  0.04631757736206055
Errors  0


In [44]:
# 100 nodes /  fanout 1
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  10009.006000427205
Total transactions  600330
Peak throughput  11002
Median round latency  0.031522154808044434
Median all received latency  0.02484571933746338
Errors  0


In [41]:
# 300 nodes /  fanout 5
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  4269.598089068042
Total transactions  256114
Peak throughput  4939
Median round latency  0.38065052032470703
Median all received latency  0.7387137413024902
Errors  0


In [38]:
# 200 nodes /  fanout 5
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  4008.943540957877
Total transactions  240421
Peak throughput  4514
Median round latency  0.2909599542617798
Median all received latency  0.5337694883346558
Errors  0


In [35]:
# 100 nodes /  fanout 5
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  3993.484810680924
Total transactions  239564
Peak throughput  4638
Median round latency  0.18499791622161865
Median all received latency  0.2911151647567749
Errors  0


In [31]:
# 200 nodes / 
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  21023.543429810477
Total transactions  635502
Peak throughput  12460
Median round latency  0.057007551193237305
Median all received latency  0.04381513595581055
Errors  0


In [208]:
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))


Est system throughput  1006.6532183960269
Total transactions  60400
Peak throughput  1086
Median round latency  0.1883605718612671
Median all received latency  0.354001522064209


In [218]:
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Num of errs ", len(errs))

Est system throughput  5825.927056511227
Total transactions  346048
Peak throughput  2750
Median round latency  0.025752544403076172
Median all received latency  nan
Num of errs  172997


/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [226]:
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Errors ", len(errs))


Est system throughput  2280.919426336533
Total transactions  136844
Peak throughput  1977
Median round latency  0.15255236625671387
Median all received latency  nan
Errors  67867


In [242]:
print("Est system throughput ", len(tx_seen)/(max_time-min_time))
print("Total transactions ", len(tx_seen))
print("Peak throughput ", max(throughput.values()))
print("Median round latency ", np.median(latency_round))
print("Median all received latency ", np.median(latency_all))
print("Num of errs ", len(errs))

Est system throughput  3487.9940403920973
Total transactions  209274
Peak throughput  2853
Median round latency  0.2164599895477295
Median all received latency  nan
Num of errs  103570


###  Experiments draft 

1. S = 15  / N = 50, 100, 200, 300 / R = 10 per node / b = 5

 N = 50
Est system throughput  2280.919426336533
Total transactions  136844
Peak throughput  1977
Median round latency  0.15255236625671387
Median all received latency  nan
Errors  67867

N = 100 

Est system throughput  3354.5847948681253
Total transactions  201252
Peak throughput  2813
Median round latency  0.22199273109436035
Median all received latency  nan
Num of errs  99596

Est system throughput  3487.9940403920973
Total transactions  209274
Peak throughput  2853
Median round latency  0.2164599895477295
Median all received latency  nan
Num of errs  103570


Est system throughput  3966.825867093246
Total transactions  237982
Peak throughput  3380
Median round latency  0.17954659461975098
Median all received latency  nan


N = 200 

Est system throughput  3209.0072227527985
Total transactions  192526
Peak throughput  3065
Median round latency  0.4524219036102295
Median all received latency  nan
Num of errs  95070

N = 300 


Est system throughput  3853.0963158375985
Total transactions  231156
Peak throughput  3312
Median round latency  0.5554780960083008
Median all received latency  nan
Num of errs  114337

N = 400 

Est system throughput  3883.6565456083445
Total transactions  233006
Peak throughput  3406
Median round latency  0.7155075073242188
Median all received latency  nan
Num of errs  115231


